# ML exercise to classify cats and dogs images

In [10]:
import nest_asyncio
nest_asyncio.apply()
import asyncio
import collections
# __import__('IPython').embed()

from ml.feature_extractor import ImageFeatureExtractor
from ml.model import ImageModel
from file_parsers import DirectoryParser, ImageParser

In [11]:
input_directory = 'demo'
dir_parser = DirectoryParser(input_directory)
print(dir_parser)

Found 374 jpg/png file(s) out of 375 file(s)


Coroutine to read images from file, resize them and transform to arrays:

In [12]:
async def main(image_parser, image_paths):
    data = collections.defaultdict(list)
    for filepath in image_paths:
        dt = await image_parser.resize(filepath)
        for key, value in dt.items():
            data[key].append(value)
    return data

Parce images with the above coroutine:

In [13]:
image_parser = ImageParser()
loop = asyncio.get_event_loop()
data = loop.run_until_complete(main(image_parser, dir_parser.full_path_image_files))
# loop.close()      

In [5]:
feature_extractor = ImageFeatureExtractor(data)
feature_extractor.print_stats()

number_of_samples: 374
description: --
image_shape: (150, 150, 3)
labels: ['cat' 'dog']
labels_distribution: {'cat': 213, 'dog': 161}


In [14]:
model = ImageModel(feature_extractor.features, feature_extractor.labels)

In [15]:
prediction = model.predict(load_from=None, save_model=False)
model.accuracy(prediction)

76.0

## Confusion Matrix

In [23]:
from sklearn.metrics import confusion_matrix
import pandas as pd
 
label_names = ['yes', 'no']
cmx = confusion_matrix(model.y_test, prediction, labels=data['label'])
df = pd.DataFrame(cmx, columns=label_names, index=label_names)
df.columns.name = 'prediction'
df.index.name = 'label'
df

ValueError: Shape of passed values is (374, 374), indices imply (2, 2)